# Assignment 1 - Mushroom Expert Chatbot
Built with HuggingFace 
Eric Marquez

### 1. First simple generation
For the means

In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor

# fairly small but good model
MODEL_NAME = "Qwen/Qwen2.5-VL-3B-Instruct"

# We're using the `Qwen2_5_VLForConditionalGeneration` class to enable multimodal generation
# Normally, you can use AutoModelForCausalLM
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    dtype="auto",  # automatically uses right precision based on model
    device_map="auto"  # automatically uses right device e.g. GPU if available
)

# We're using the `AutoProcessor` class to enable multimodal generation
# Normally, you can use AutoTokenizer
processor = AutoProcessor.from_pretrained(MODEL_NAME)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [2]:
from PIL import Image

from transformers import GenerationConfig
from qwen_vl_utils import process_vision_info

from transformers import TextStreamer


In [3]:
DATA_PATH = "./data"
#IMAGE_PATH = "./data/engineer_fitting_prosthetic_arm.jpg"
#"work/data/engineer_fitting_prosthetic_arm.jpg"
#im = Image.open(IMAGE_PATH)

# Your code here
 

In [4]:
import re

def parse_mushroom_file(filepath: str) -> dict:
    data = {}
    current_key = None
    current_lines = []

    # Regex to detect filename keys (with or without colon)
    key_pattern = re.compile(r"^.+\.(jpg|jpeg|png|gif):?$", re.IGNORECASE)

    with open(filepath, encoding="utf-8") as f:
        for line in f:
            line = line.rstrip("\n")

            if key_pattern.match(line.strip()):
                # Save the previous entry if any
                if current_key is not None:
                    data[current_key] = " ".join(current_lines).strip()

                # Normalize key (remove trailing colon if present)
                current_key = line.strip().rstrip(":")
                current_lines = []
            elif line.strip():  # non-empty description line
                current_lines.append(line.strip())

        # Save the last entry
        if current_key is not None:
            data[current_key] = " ".join(current_lines).strip()

    return data


In [14]:
mushrooms_dict = parse_mushroom_file(f"{DATA_PATH}/mushrooms.txt")

for k, v in mushrooms_dict.items():
    print(k, "->", v[:60] + "...")


mushroom_copper_spike.jpg -> A picture of two copper spike mushrooms growing in the fores...
mushroom_deadly_webcap.jpg -> A picture of five deadly webcap mushrooms growing in the for...
mushroom_false_chanterelle.jpg -> A picture of three false chanterelle mushroom growing in the...
mushroom_parasol_topview.jpg -> A picture of a false parasol mushroom growing on grass. The ...


In [17]:
#To build the conversation as a list of dictionaries
conversation = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": "You are a mushroom expert chatbot that responds to user queries about mushrooms, and you must always steer the conversation to keep it in the context of mushrooms. You provide succinct and to the point information about mushrooms based on data. You talk in mycological terms about data. You use the information given to you from the data and answers to earlier user/assistant pairs to provide information about mushrooms. "}
        ],
    },
]

#images_dict = {"type": "image", "image": "image path here"}
user_prompt_str = "Tell me the name of the mushroom or mushrooms in this picture. Tell me what type of mushrooms are. Give me their physical description. Clarify if they are edible and if not, explain why and what are the sympthoms or side effects in a person, in this last case explain if there is a known treatment or antidote. Mention if there is any similarity with other specties. Lastly, you may mention any other documented fact about the mushroom."
#text_descr_dict = {"type": "text", "text": "text description of the picture here"}
#user_prompts_dict = {"type": "text", "text": user_prompt_str }
#content_user_list = [] #"List of images(dict) and its respectives users prompts(dict) here"
#role_user_dict ={"role": "user", "content": "content_user_list here"}
#role_assistant_dict = {"role": "assistant", "content": "content_assistant_list here"}

for k, v in mushrooms_dict.items():
    images_dict = {"type": "image", "image": f"{DATA_PATH}/{k}"}    
    user_prompts_dict = {"type": "text", "text": user_prompt_str }
    content_user_list = [images_dict, user_prompts_dict]
    role_user_dict ={"role": "user", "content": content_user_list}
    text_descr_dict = {"type": "text", "text": v}
    role_assistant_dict = {"role": "assistant", "content": [text_descr_dict]}
    
    conversation.append(role_user_dict)
    conversation.append(role_assistant_dict)

print (conversation)

[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a mushroom expert chatbot that responds to user queries about mushrooms, and you must always steer the conversation to keep it in the context of mushrooms. You provide succinct and to the point information about mushrooms based on data. You talk in mycological terms about data. You use the information given to you from the data and answers to earlier user/assistant pairs to provide information about mushrooms. '}]}, {'role': 'user', 'content': [{'type': 'image', 'image': './data/mushroom_copper_spike.jpg'}, {'type': 'text', 'text': 'Tell me the name of the mushroom or mushrooms in this picture. Tell me what type of mushrooms are. Give me their physical description. Clarify if they are edible and if not, explain why and what are the sympthoms or side effects in a person, in this last case explain if there is a known treatment or antidote. Mention if there is any similarity with other specties. Lastly, you may mention any 

In [8]:
conversation = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": "You are a mushroom expert chatbot that responds to user queries about mushrooms, and you must always steer the conversation to keep it in the context of mushrooms. You provide succinct and to the point information about mushrooms based on data. You talk in mycological terms about data. You use the information given to you from the data and answers to earlier user/assistant pairs to provide information about mushrooms. "}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "image_file"},        
            {"type": "text", "text": "Tell me: Based on your expertise, what could had happen to the person in this picture to reach that state?"}
        ],
    },
]

In [9]:
print(conversation)

[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a mushroom expert chatbot that responds to user queries about mushrooms, and you must always steer the conversation to keep it in the context of mushrooms. You provide succinct and to the point information about mushrooms based on data. You talk in mycological terms about data. You use the information given to you from the data and answers to earlier user/assistant pairs to provide information about mushrooms. '}]}, {'role': 'user', 'content': [{'type': 'image', 'image': 'image_file'}, {'type': 'text', 'text': 'Tell me: Based on your expertise, what could had happen to the person in this picture to reach that state?'}]}]
